<a href="https://colab.research.google.com/github/KellieChong/REE-ED-Model/blob/main/REE_ED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the required libraries
import numpy as np
import math
import ipywidgets as widgets
from ipywidgets import fixed, interact, interactive, Layout
from IPython.display import display
from scipy.optimize import fsolve, root
from scipy.integrate import RK45, odeint, solve_ivp
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files

In [ ]:
# let us define some units for the various variables used in the calculations

#==============================================
# Symbol | Meaning                | Units 
#==============================================
# (C)    | Concentrations         | [mol/m3]
# (D)    | Diffusion coefficient  | [m2/s]
# (F)    | Faraday constant       | [C/mol]
# (I)    | Current density        | [A/m2]
# (J)    | Flux in membrane       | [mol/(m2*s)]
# (Kabs) | Absolute stab. constant| [dm3/mol]
# (Q)    | Ion-exchange capacity  | [mol/m3]
# (S)    | Surface area           | [m2]
# (Si)   | Separation factor      | [unitless]
# (t)    | Time                   | [s]
# (V)    | Volume of solution     | [m3]

#for EDTA, HEDTA, and DCTA columns of the matrix, they are the K_abs values, while columns 3 and 4 are diffusion const and K_H respectively
chelant_hash = {"EDTA": 0, "HEDTA": 1, "DCTA":2, "D":3, "K_H":4} # the columns of the database matrix
element_hash = {"La": 0, "Pr": 1, "Nd": 2, "Gd": 3, "Y": 4, "dissConst": 5, "stoichiometric ratio": 6} # rows of the database matrix
#database = np.zeros((len(element_hash), len(chelant_hash)))
dcEDTA = np.log10([2.00, 2.67, 6.16, 10.26])
dcDCTA = np.log10([2.40, 3.55, 6.14, 10.26])
dcHEDTA = np.log10([3.23, 5.50, 10.02])
#map all the values to the database matrix now
database = [
    [10**15.50, 10**13.22, 10**16.60, 4.4e13, 1.49], #La
    [10**16.40, 10**14.39, 10**17.01, 4.8e13, 1.40], #Pr
    [10**16.61, 10**14.71, 10**17.31, 5.2e13, 1.15], #Nd
    [10**17.37, 10**15.10, 10**18.70, 6.5e13, 0.91], #Gd
    [10**18.09, 10**14.49, 10**19.60, 11.0e13, 0.66], #Y
    [dcEDTA, dcHEDTA, dcDCTA, 0, 0], #dissociation constants
    [0.5, 1/3, 0.5, 0, 0] # CA:Na stoichiometric ratio
             ]

mass = {"EDTA": 292.24, "HEDTA": 344.2, "DCTA": 364.35, "La": 138.905 , "Pr": 140.907, "Nd": 144.242, "Gd": 157.25, "Y": 88.90585}

# some other contants used in the model/experiment
memSA = 0.00423
zREE = 3
fConst = 96485
volume = 0.0013
I = 20 #0.0312 #limiting current density
F = 96485

#---------------------------- Main function starts here to code for each of the 3 sections of Takhashi's paper -------------------------
        
def main (LREE, HREE, chelant, C_light, C_heavy, totChelantConc, pH, t, Q, n, constantH, solver):
    
    #pack init_conds
    init_conds = [LREE, HREE, chelant, C_light, C_heavy, totChelantConc, pH, t, Q, n, constantH, solver] 
    init_conds[3] = C_light / mass[LREE] #convert concentrations (ppm) from mol/L to mol/m3
    init_conds[4] = C_heavy / mass[HREE] 
    init_conds[5] = init_conds[4] * totChelantConc
    print('\nStarting concentrations (mol/m3): ', str(init_conds[3:6]))
    #Set the path on the tree for the correct constants based on the user's input
    #compute the concentration of hydrogen based off the sample's pH
    C_H = 10 ** -pH * 1000 # convert pH to mol/m3
    K_abs_LREE = database[element_hash[LREE]][chelant_hash[chelant]]
    K_abs_HREE = database[element_hash[HREE]][chelant_hash[chelant]]
    dissConst = database[element_hash["dissConst"]][chelant_hash[chelant]]
    K_H_light = database[element_hash[LREE]][chelant_hash["K_H"]]
    K_H_heavy = database[element_hash[HREE]][chelant_hash["K_H"]]
    eqv = database[element_hash["stoichiometric ratio"]][chelant_hash[chelant]]
    tspan = np.linspace (0, t, n)
    Q_converted = Q*1000 #Q is now converted to mol/m3
    res  = solutions(init_conds, C_H, K_abs_LREE, K_abs_HREE, dissConst, K_H_light, K_H_heavy, Q_converted, eqv)
    C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion, SF = [np.array(sol) for sol in res]
    
    fig = plot(LREE, HREE, chelant, constantH, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion, tspan, SF)
    
    #uncomment if you want to save solutions to a csv file
    #exportData(tspan, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion)
    
    return fig, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion, SF 

# 1) Dissociation equilibrium of solutions on first iteration:
def dissEquilibrium(xout, chelant, constantH, pH, totChelantConc, dissConst, K_abs_LREE, K_abs_HREE, C_light, C_heavy):
    if constantH == True:
        #the order for the xs are: x1:C_light_ion, x2:C_lightComplex x3:C_heavy_ion, x4:C_HeavyComplex, x5:C_noComplex x6:C_Na, x7 = C_H 
        x1, x2, x3, x4, x5, x6 = xout #C_Na is calculated for when [H] is const
        x7 = 10 **-pH * 1000 # concentration must be converted to mol/m3 still
    else:
        x1, x2, x3, x4, x5, x7 = xout
        x6 = totChelantConc #here C_Na = x6 = totChelantConc
        
    alphaDenom  = np.prod(dissConst[:]) + np.prod(dissConst[0:2])*x7 +\
        np.prod(dissConst[:1])*x7**2 +\
            np.prod(dissConst[0]* x7**3) + \
                      x7**4
    if chelant == "HEDTA":
         alphaDenom  = np.prod(dissConst[:]) + np.prod(dissConst[0:1])*x7 +\
              np.prod(dissConst[0])*x7**2 +\
                      x7**3
  
    y = [K_abs_LREE * x1 * x6 - x2, 
          K_abs_HREE * x3 * x6 - x4,
          C_light - x1 - x2,
          C_heavy - x3 - x4,
          totChelantConc - x2 - x4 - x5,
          x6 - np.prod(dissConst[0:-1])/alphaDenom * x5]
    
    return y
    
# 2) Ion exchange of membranes
def ionExchange(memConc, K_H_light, K_H_heavy, C_lightion, C_heavyion, C_Na, C_H,  Q_converted ):
    
    eq = [memConc[0] - K_H_light * memConc[3] **3 *(C_lightion/C_H**3)**0.8,
          memConc[1] - K_H_heavy * memConc[3] **3 *(C_heavyion/C_H**3)**0.8,
          memConc[2] - 0.32 * memConc[3] *(C_Na/C_H),
          Q_converted - 3 * (memConc[0] + memConc[1] + memConc[2] + memConc[3])]

    return eq

# 3) permeation  
def permeation(t, Cions_sol, LREE, HREE, C_LightMem, C_HeavyMem, C_Na_mem, C_H_mem, constantH):
  
    C_Light, C_Heavy, C_Na, C_H = Cions_sol
    
    commonDenom = zREE**2 * database[element_hash[LREE]][chelant_hash["D"]] * C_LightMem + zREE**2 * database[element_hash[HREE]][chelant_hash["D"]] * \
                    C_HeavyMem + 1.9e15 * C_Na_mem + 1.1e16 * C_H_mem
    
    dCdt_Light = -memSA * zREE * database[element_hash[LREE]][chelant_hash["D"]] * C_LightMem/commonDenom * I/(F*volume)
    dCdt_Heavy = -memSA * zREE * database[element_hash[HREE]][chelant_hash["D"]] * C_HeavyMem/commonDenom * I/(F*volume)
    dCdt_Na = -memSA * 1.9e15 * C_Na_mem/commonDenom * I/(F*volume)
    dCdt_H = -memSA * 1.1e16 * C_H_mem/commonDenom * I/(F*volume)
    if constantH == 1:
      dCdt_H = 0
    return [dCdt_Light, dCdt_Heavy, dCdt_Na, dCdt_H]


In [ ]:
def solutions(init_conds, C_H, K_abs_LREE, K_abs_HREE, dissConst, K_H_light, K_H_heavy, Q_converted, eqv):    
  
    LREE, HREE, chelant, C_light, C_heavy, totChelantConc, pH, t, Q, n, constantH, solver = init_conds #unpack elements
    
    D_light = database[element_hash[LREE]][chelant_hash["D"]]
    D_heavy = database[element_hash[HREE]][chelant_hash["D"]]

    #set up the time span for the size of the mactrices
   
    tspan = np.linspace (0, t, n)
    
    #create the matrices for solutions to later be plotted
    C_light_ion = [] #x1
    C_heavy_ion  = [] #x3
    C_Na_ion = [] #x6
    C_H_ion = [] #x7 if C is changing
    SF = []

    # dissociation equilibrium only needs to be run once, outside the loop
    if constantH == True:
          C0ion = [C_light * 0.6, C_light *  0.1, C_heavy * 0.6, C_heavy * 0.1, totChelantConc * 0.5 , totChelantConc] #last guess is for C_Na
    else:
        C0ion = [C_light*0.6, 0.001, C_heavy*0.6, 0.001,  totChelantConc*0.1, C_H * 0.6] #last value is for C_H
    
    xout, infodict, ier, mesg = fsolve(dissEquilibrium, C0ion, args=(chelant, constantH, totChelantConc, pH, dissConst, K_abs_LREE, K_abs_HREE, C_light, C_heavy), full_output=1)
    #xout = root(dissEquilibrium, C0ion, args=(constantH, totChelantConc, pH, dissConst, K_abs_LREE, K_abs_HREE, C_light, C_heavy))
    print("\nDissociation Equilibrium solutions: \n", xout, mesg, "\n")
    #xout = np.array(xout)
    #print(mesg)
    C_lightion, C_lightComplex, C_heavyion, C_heavyComplex, C_noComplex = xout[0:5]  # do not append to matrix, it will be overwritten after section 3, just store as a temporary guess
    if constantH == False:
        C_H = xout[-1] #C_H if the concentration of H is changing
        C_Na = totChelantConc / eqv
    else:
        #do not need to redefine C_H as it is passed into the function already 
        C_Na = xout[-1] / eqv #remember that xout[5] is C_Na when C_H is not changing
        
    
    #Now let us run the functions and iterate over them
    for i in range (0, n):
      #set initial guesses
      if i == 0:
        C0mem = [C_lightion*0.7, C_heavyion*0.7, C_Na *0.5, C_H *0.5]
        Ci0 = [C_lightion , C_heavyion , C_Na, C_H]
        #the first iteration is kind of different s
      else:
        C0mem = [C_LightMem_sol, C_HeavyMem_sol, C_Na_mem_sol, C_H_mem_sol]
        #inital guess for permeation is the same no matter what iteration
        Ci0 = [C_lightion, C_heavyion, C_Na, C_H]
        
      #print("\nt(s) = ", str(i*(t/n)))
  
      #Section 2: Ion exchange:
      memConc, infodict, ier, mesg = fsolve(ionExchange, C0mem, args=(K_H_light, K_H_heavy, C_lightion, C_heavyion, C_Na, C_H,  Q_converted ), full_output = 1)
      C_LightMem_sol, C_HeavyMem_sol, C_Na_mem_sol, C_H_mem_sol = memConc
      #print(memConc, mesg) #for some reason, the ion exchange equilibrium system of equations is failing to converge

      #Section 3: Permeation equations:
      Cions_sol = solve_ivp(permeation, [tspan[i-1], tspan[i]], Ci0,  rtol = 1e-6, method = solver, args = (LREE, HREE, C_LightMem_sol, C_HeavyMem_sol, C_Na_mem_sol, C_H_mem_sol, constantH), t_eval = [tspan[i]])
      
      #Squeeze negative solutions to 0
      i = 0
      while i < 4:
        if float(Cions_sol.y[i]) < 0:
          Cions_sol.y[i] = 0
        i+=1
      #print(Cions_sol.y, Cions_sol.success)
      #redefine C_lightion, C_heavyion, C_Na, and C_H now to be used in next iteration:
      C_lightion = Cions_sol.y[0][-1]
      C_heavyion = Cions_sol.y[1][-1]
      C_Na = Cions_sol.y[2][-1]
      if constantH == False:
        C_H = Cions_sol.y[3][-1]
        C_H_ion.append(C_H)
      else:
        C_H_ion.append(C_H)

      #append for the final solution
      C_light_ion.append(C_lightion)
      C_heavy_ion.append(C_heavyion)
      C_Na_ion.append(C_Na*eqv)
      
      #Calculating the separation factor now
      SF.append(C_lightion/C_heavyion)
    
    return C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion, SF


def plot(LREE, HREE, chelant, constantH, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion, tspan, SF):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))
    #ax1 = fig.add_subplot()
    ax1.set_title("Concentration Over Time", FontSize=16)
    ax1.set_xlabel("Time(s)", FontSize=12)
    ax1.set_ylabel("Ion Concentration (mol/L)", FontSize=12)
   
    ax1.plot(tspan, np.array(C_light_ion)/1000, label = '[%s3+](t)' % LREE, color='orange') #note that you must divide the concentrations by 1000 to convert mol/m3 to mol/L (1000 L = 1 m3)
    ax1.plot(tspan, np.array(C_heavy_ion)/1000, label = '[%s3+](t)' % HREE, color='violet')
    ax1.plot(tspan, np.array(C_Na_ion)/1000, label = '[%s](t)' % chelant, color='mediumseagreen')
    ax1.text(0, 0, "The Separation factor is: " + str(SF[-1]) , bbox=dict(facecolor='lightskyblue', alpha=0.5),  FontSize=15)
    if constantH == False:
        ax1.plot(tspan, np.array(C_H_ion)/1000, label = "[H+](t)")
  
    ax1.legend(loc = 'best', prop={'size': 12})
    if SF[-1] > 40:
      ax2.set_ylim([0, 60])
    ax2.set_title("Separation Factor Over Time", FontSize=16)
    ax2.set_xlabel("Time(s)", FontSize=12)
    ax2.set_ylabel("Separation Factor", FontSize=12)
    ax2.plot(tspan, SF, label="%s / %s SF" %(LREE, HREE), color='deeppink')
    #the first index is the total experiment time
    batch23152 = [18000, 0, 1.637573269, 2.278953099, 2.468467156, 2.798070427, 2.701350884, 1.067955093, 2.717399739, "batch 23152"] #batch 23152 (La-Nd-825-825-1:1-DCTA)
    batch23110 = [14400, 0, 2.27, 2.60, 2.80, 3.35, 2.85, 2.51, 2.70, "batch 23110"] #La-Nd-825-825-2:1-EDTA-t=240 min (14400 s)
    batch23127 = [18000, 1.939130974, 3.413435218, 1.939319398, 1.980772263, 4.379358996, 2.778676488, 4.701797055, "batch 23127"] #La-Nd-825-825-2:1-HEDTA-300 min
    batch23990 = [18000, 1.095213587, 1.291514406, 1.480146147, 1.608031404, 1.748234974, 1.948776634, 2.075875038, 2.241259614, "batch 23990"] #La-Nd-825-1030-2:1-HEDTA-300 min
    
    if LREE == 'La' and HREE == 'Nd':
        if chelant == "EDTA":
          y = batch23110
        elif chelant == "DCTA":
          y = batch23152
        else:
          y = batch23127
          ax2.plot(np.linspace(1, batch23990[0], len(batch23990)-2), batch23990[1:len(batch23990)-1], label = "batch 23990", color='cornflowerblue')
        ax2.plot(np.linspace(1, y[0], len(y)-2), y[1:len(y)-1], label = y[-1], color='darkviolet')
        ax2.text(0, 0, "Experimental Separation factor: " + str(y[-2]) , bbox=dict(facecolor='lightskyblue', alpha=0.5),  FontSize=15)
    ax2.legend(loc="best", prop={'size': 12})
    return fig


def exportData(tspan, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion):
  #this function is to save solutions to a csv file
    headers = ["Time (s)", "LREE", "HREE", "Na+", "H+"]
    
    data = np.reshape([tspan, C_light_ion, C_heavy_ion, C_Na_ion, C_H_ion], (len(tspan), 5))
    datafile = pd.DataFrame(data = data, columns=headers)
    datafile.to_csv("La-Nd-825-825-EDTA-1-1.csv", index=False)
    files.download("La-Nd-825-825-EDTA-1-1.csv") 
    
    return


In [ ]:
def gui():
    slider_style = {'description_width': 'initial', 'handle_color': 'lightblue'}
    # This section codes for the GUI (front-end)
    LREE = widgets.Dropdown(options = ["La", "Pr", "Nd", "Gd", "Y"], value ="La", description = "LREE", continuous_update = True)
    HREE = widgets.Dropdown(options = ["La", "Pr", "Nd", "Gd", "Y"], value ="Nd", description = "HREE", continuous_update = True)    
    chelant = widgets.Dropdown(options=["EDTA", "DCTA", "HEDTA"], value = "EDTA", description = "Chelating agent", continuous_update = True, style={'description_width': 'initial'})
    C_light = widgets.FloatSlider(min = 0.0, value = 825, max = 1500, description = "[LREE] ppm", step = 5, continuous_update = True, style=slider_style)
    C_heavy = widgets.FloatSlider(min = 0.0, value = 825, max = 1500, description = "[HREE] ppm ", step = 5, continuous_update = True, style=slider_style)
    totChelantConc = widgets.FloatSlider(min = 0, max = 5.00, value = 1, description = "CA:HREE ratio", step = 0.5, continuous_update = True, style=slider_style)
    pH =widgets.FloatSlider(min = 0, value = 4.5, max = 14.0, description = "pH", continuous_update=1, style=slider_style)
    t = widgets.IntSlider(min = 600, max = 36000, value='18000', step = 1, description = "Duration(s)", style=slider_style)
    Q = widgets.FloatSlider(min = 0, step = 0.5, value = 1.475, max = 10, description = "Ion-Exchange Capacity (mEq/g)", style=slider_style)
    n = widgets.IntSlider(min = 10, max = 500, value=250, step = 1, description = "Number of datapoints in solution", style=slider_style)
    constantH = widgets.Checkbox(value = True, description='Constant [H+]')
    solver = widgets.ToggleButtons(options = ["RK45", "DOP853","LSODA"], value = "RK45", description = "Numerical Method Selection", button_style = 'success', disabled = False, icons=['check'], style={'description_width': 'initial'})
    
    init_conds = [LREE, HREE, chelant, C_light, C_heavy, totChelantConc, pH, t, Q, n, constantH, solver]
    
    out = widgets.interactive_output(main, {"LREE": LREE, "HREE": HREE, "chelant": chelant, "C_light": C_light, "C_heavy": C_heavy, "totChelantConc":totChelantConc, \
                                        "pH": pH,"t": t, "Q": Q, "n": n, "constantH": constantH, "solver": solver})
    R1 = widgets.HBox([LREE, HREE, chelant, pH, constantH, solver], Layout="scroll hidden")
    R2 = widgets.HBox([C_light, C_heavy, totChelantConc, Q, t, n], Layout='scroll hidden')
    tab1 = widgets.VBox([R1, R2, out])
    
    tab = widgets.Tab([tab1])
    tab.set_title(0, "Concn Over Time Plot")
    gui = widgets.VBox([tab])
    
    items_layout = widgets.Layout(width = 'auto')
    box_layout = widgets.Layout (display = 'flex-flow', flex_flow = 'row', align_items = 'stretch', border = 'solid', width = '100%')
    
    disp = display(gui) 

    return disp

    
gui()


In [ ]:
#Change these input values as you wish to test out specific scenaries
sol = main("La", "Nd", "HEDTA", 0.005, 0.005, 0.005, 4.50, 1800, 1.48, 300, 1, RK45)
print("\nMaximum Separation Factor", np.amax(np.array(sol[1])/np.array(sol[2])))
#print(sol[1]) #LREE ion solutions
#print(sol[2]) #HREE ion solutions
